# This is a fork of:
https://www.kaggle.com/hengck23/augmentation-using-image-blending
and uses cots-masks from 
https://www.kaggle.com/alexandrecc/cots-masks

Some changes are needed like style transfer to get best results. 

# Augmentation using image blending

- The images used for illustration purposes only. I just downloaded some creative commons license images from google image search. I haven't checked their licenses from the source website yet ... you are advised to **use your own images ** after checking their license !!!!

- Alternatively, you can just use kaggle train images and transfer COTS objects from one frame to another frame


In [ ]:
import numpy as np 
import pandas as pd  
import cv2
import random

%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
#blending algorithms


# 1. simple color transfer by rgb normalisation
#https://github.com/chia56028/Color-Transfer-between-Images/blob/master/color_transfer.py

def norm_color_transfer(src, dst):

    def get_mean_and_std(x):
        x_mean, x_std = cv2.meanStdDev(x)
        x_mean = np.hstack(np.around(x_mean,2)).reshape(1,1,3)
        x_std = np.hstack(np.around(x_std,2)).reshape(1,1,3)
        return x_mean, x_std

    s = cv2.cvtColor(src,cv2.COLOR_BGR2LAB)
    t = cv2.cvtColor(dst,cv2.COLOR_BGR2LAB)
    s_mean, s_std = get_mean_and_std(s)
    t_mean, t_std = get_mean_and_std(t)
    
    # print(s_mean, s_std, t_mean, t_std)

    m = (s-s_mean)*(t_std/s_std)+t_mean
    m = np.round(m)
    m = np.clip(m,0,255).astype(np.uint8)

    m = cv2.cvtColor(m,cv2.COLOR_LAB2BGR)
    return m




# 2. deep blending (in progress)
# https://github.com/owenzlz/DeepImageBlending




# 3. piosson editing  
# https://github.com/PPPW/poisson-image-editing
import scipy.sparse
from scipy.sparse.linalg import spsolve


def laplacian_matrix(n, m):
    """Generate the Poisson matrix.
    Refer to:
    https://en.wikipedia.org/wiki/Discrete_Poisson_equation
    Note: it's the transpose of the wiki's matrix
    """
    mat_D = scipy.sparse.lil_matrix((m, m))
    mat_D.setdiag(-1, -1)
    mat_D.setdiag(4)
    mat_D.setdiag(-1, 1)

    mat_A = scipy.sparse.block_diag([mat_D] * n).tolil()

    mat_A.setdiag(-1, 1*m)
    mat_A.setdiag(-1, -1*m)

    return mat_A


def poisson_edit(source, target, mask, offset=(0,0)):
    """The poisson blending function.
    Refer to:
    Perez et. al., "Poisson Image Editing", 2003.
    """

    # Assume:
    # target is not smaller than source.
    # shape of mask is same as shape of target.
    y_max, x_max = target.shape[:-1]
    y_min, x_min = 0, 0

    x_range = x_max - x_min
    y_range = y_max - y_min

    M = np.float32([[1,0,offset[0]],[0,1,offset[1]]])
    source = cv2.warpAffine(source,M,(x_range,y_range))

    mask = mask[y_min:y_max, x_min:x_max]
    mask[mask != 0] = 1
    #mask = cv2.threshold(mask, 127, 1, cv2.THRESH_BINARY)

    mat_A = laplacian_matrix(y_range, x_range)

    # for \Delta g
    laplacian = mat_A.tocsc()

    # set the region outside the mask to identity
    for y in range(1, y_range - 1):
        for x in range(1, x_range - 1):
            if mask[y, x] == 0:
                k = x + y * x_range
                mat_A[k, k] = 1
                mat_A[k, k + 1] = 0
                mat_A[k, k - 1] = 0
                mat_A[k, k + x_range] = 0
                mat_A[k, k - x_range] = 0

    # corners
    # mask[0, 0]
    # mask[0, y_range-1]
    # mask[x_range-1, 0]
    # mask[x_range-1, y_range-1]

    mat_A = mat_A.tocsc()

    mask_flat = mask.flatten()
    for channel in range(source.shape[2]):
        source_flat = source[y_min:y_max, x_min:x_max, channel].flatten()
        target_flat = target[y_min:y_max, x_min:x_max, channel].flatten()

        #concat = source_flat*mask_flat + target_flat*(1-mask_flat)

        # inside the mask:
        # \Delta f = div v = \Delta g
        alpha = 1
        mat_b = laplacian.dot(source_flat)*alpha

        # outside the mask:
        # f = t
        mat_b[mask_flat==0] = target_flat[mask_flat==0]

        x = spsolve(mat_A, mat_b)
        #print(x.shape)
        x = x.reshape((y_range, x_range))
        #print(x.shape)
        x[x > 255] = 255
        x[x < 0] = 0
        x = x.astype('uint8')
        #x = cv2.normalize(x, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
        #print(x.shape)

        target[y_min:y_max, x_min:x_max, channel] = x
    return target

In [ ]:
#helper
def make_blend_mask(size, object_box):
    x,y,w,h = object_box
    x0=x
    x1=x+w
    y0=y
    y1=y+h


    w,h = size
    mask = np.ones((h,w,3),np.float32)

    for i in range(0,y0):
        mask[i]=i/(y0)
    for i in range(y1,h):
        mask[i]=(h-i)/(h-y1+1)
    for i in range(0,x0):
        mask[:,i]=np.minimum(mask[:,i],i/(x0))
    for i in range(x1,w):
        mask[:,i]=np.minimum(mask[:,i],(w-i)/(w-x1+1))

    return mask


def insert_object(mix, box, crop, mask):
    x,y,w,h = box
    crop = cv2.resize(crop, dsize=(w,h), interpolation=cv2.INTER_AREA)
    mask = cv2.resize(mask, dsize=(w,h), interpolation=cv2.INTER_AREA)

    mix_crop = mix[y:y+h,x:x+w]
    crop = norm_color_transfer(crop, mix_crop)
    mix[y:y+h,x:x+w] = mask*crop +(1-mask)*mix_crop
    return mix

In [ ]:
import os
from tqdm import tqdm

fns = os.listdir('/kaggle/input/cots-masks/images')

total_objects = []

for fn in tqdm(fns):
    image = cv2.imread('../input/cots-masks/images/' + fn, cv2.IMREAD_COLOR)
    mask =  cv2.imread('../input/cots-masks/masks/' + fn[:-3] + 'png', cv2.IMREAD_COLOR)

    total_objects.append([image, mask])

In [ ]:
df = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
df = df[df['annotations'] == '[]']

def get_rand_background():
    global df
    
    row = df.sample()
    
    image = cv2.imread(f'../input/tensorflow-great-barrier-reef/train_images/video_{row.video_id.values[0]}/{row.video_frame.values[0]}.jpg', cv2.IMREAD_COLOR)
#     print(f'../input/tensorflow-great-barrier-reef/train_images/video_{row.video_id.values[0]}/{row.video_frame.values[0]}.jpg')

    if random.random() > .5:
        image = cv2.flip(image, 0)
        
    if random.random() > .5:
        image = cv2.flip(image, 1)
    
    return image
    

In [ ]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [ ]:
!rm -rf labels
!rm -rf images

!mkdir images
!mkdir labels

In [ ]:
kernel = np.ones((5,5),np.float32)/25

# cv2.rectangle(mix1, (x-50,y-50), (x+w+50,y+h+50), (255,255,255), 2)

# mix = background.copy()


for im_id in tqdm(range(10000)):
    mix = get_rand_background()
    bboxes = []
    for i in range(5):
        crop, mask = total_objects[i]

        x, y = random.randint(int(1280 * .1), int(1280 * .9)), random.randint(int(720 * .1), int(720 * .9))

        # random resize
        hi = random.randint(15, 100)
        crop = image_resize(crop, height=hi)
        mask = image_resize(mask, height=hi)


        # w = random.randint(20, 100)
        # h = max(20, w - random.randint(20, 30))

    #     crop = cv2.resize(crop, dsize=(h,w), interpolation=cv2.INTER_AREA)
    #     mask = cv2.resize(mask, dsize=(h,w), interpolation=cv2.INTER_AREA)


        mask = cv2.filter2D(mask,-1,kernel)

        # augmentations
        if random.random() > .5:
            crop = cv2.flip(crop, 0)
            mask = cv2.flip(mask, 0)

        if random.random() > .5:
            crop = cv2.flip(crop, 1)
            mask = cv2.flip(mask, 1)

        if random.random() > .5:
            crop = cv2.rotate(crop, cv2.ROTATE_90_COUNTERCLOCKWISE)
            mask = cv2.rotate(mask, cv2.ROTATE_90_COUNTERCLOCKWISE)

        if random.random() > .5:
            crop = cv2.rotate(crop, cv2.ROTATE_180)
            mask = cv2.rotate(mask, cv2.ROTATE_180)

        if random.random() > .5:
            crop = cv2.rotate(crop, cv2.ROTATE_90_CLOCKWISE)
            mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)

        h, w, _ = mask.shape

    #     mask = np.float32(mask)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

        nmask = np.zeros((h, w, 3))
        nmask[:, :, 0] = mask
        nmask[:, :, 1] = mask
        nmask[:, :, 2] = mask

    #     print(h,w)

        nmask = (nmask/255) * random.uniform(.7, .95)
        # nmask =nmask

        mix_crop = mix[y:y+h,x:x+w]
        crop = norm_color_transfer(crop, mix_crop) 
    #     crop = poisson_edit(crop, mix_crop, (mask).astype(np.float32), offset=(0,0))

        mix[y:y+h,x:x+w] = nmask*crop +(1-nmask)*mix_crop
        
        # blur top of it 
        # mix[y-int(h * .10):y+int(h * .10),x:x+w] = cv2.filter2D(mix[y-int(h * .10):y+int(h * .10),x:x+w],-1,kernel)

        bboxes.append([x, y, w, h])

        # cv2.rectangle(mix, (x-20,y-20), (x+w+20,y+h+20), (255,255,255), 2)
    # Save image
    cv2.imwrite(f'./images/{im_id}.jpg', mix)
    
    with open(f'./labels/{im_id}.txt', 'w') as f:
        for x, y, w, h in bboxes:
            f.write(f'0 {x / 1280} {y / 720} {w / 1280} {h / 720}\n')

    

In [ ]:
import shutil
shutil.make_archive('labels', 'zip', 'labels')

import shutil
shutil.make_archive('images', 'zip', 'images')

In [ ]:
!rm -rf images
!rm -rf labels

In [ ]:
plt.figure()
plt.imshow(mask[...,::-1])

plt.figure(figsize=(25,25))
plt.imshow(mix[...,::-1])